In [5]:
import pandas as pd
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import locale
import win32com.client as win32

servico = Service(ChromeDriverManager().install())
navegador = webdriver.Chrome(service=servico)

In [6]:
#Carregar a tabela de buscas de produtos:
produtos_df = pd.read_excel(r'Base de Buscas\buscas.xlsx')
display(produtos_df)

,Nome,Termos banidos,Preço mínimo,Preço máximo
0,iphone 12 64gb,mini watch 11 xr,3000,3500
1,rtx 3060,zota galax,4000,4500


In [7]:
def formatar_moeda(numero):
    '''Formata os valores para o padrão de moeda PT-BR'''
    locale.setlocale(locale.LC_ALL, 'pt_BR.UTF-8')
    return locale.currency(numero, grouping=True, symbol=None)

def buscar_precos_google_shopping(navegador, nome, termos_banidos, preco_minimo, preco_maximo):
    #Abrir o site do Google Shopping:
    navegador.get("https://shopping.google.com.br/")
    #Buscar o produto:
    navegador.find_element(By.ID, 'REsRA').send_keys(nome)
    navegador.find_element(By.CLASS_NAME, 'uDgUL').click()
    #Definir preços:
    preco_minimo = float(preco_minimo)
    preco_maximo = float(preco_maximo)
    #Crias lista de filtros da busca:
    lista_termos_produto = nome.lower().split(" ")
    lista_termos_banidos = termos_banidos.lower().split(" ")
    
    #Lista que será retornada pela função:
    lista_ofertas_google = []

    while True:
        #Esperar até que os resultados da pesquisa estejam disponíveis na página
        WebDriverWait(navegador, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'sh-dgr__content')))
        time.sleep(1)
        
        #Pegar todos os produtos da página:
        lista_produtos = navegador.find_elements(By.CLASS_NAME, 'sh-dgr__content')
        
        for produto in lista_produtos:
            try:
                #Obter os dados dos produtos:
                nome_produto = produto.find_element(By.TAG_NAME, 'h3').text.lower().replace(',','')
                preco_produto = produto.find_element(By.CLASS_NAME, 'a8Pemb').text.replace(' ', '').replace('R$', '').replace('.', '').replace(',', '.')
                
                #Filtrar pelos dados da busca:
                termo_prod_correto = all(termo in nome_produto for termo in lista_termos_produto)
                termo_banido_encontrado = any(termo in nome_produto for termo in lista_termos_banidos)

                if termo_prod_correto and not termo_banido_encontrado:
                    preco_produto = float(preco_produto)
                    if preco_minimo <= preco_produto <= preco_maximo:
                        preco_produto = formatar_moeda(preco_produto)
                        link_produto = produto.find_element(By.TAG_NAME, 'a').get_attribute('href')
                        link_produto = link_produto.replace('https://www.google.com.br/url?url=', '')
                        lista_ofertas_google.append((nome_produto, preco_produto, link_produto))
            except:
                pass
            
        try:
            # Tentar clicar no botão "Mais"
            proxima_pag = navegador.find_element(By.LINK_TEXT, 'Mais')
            proxima_pag.click()
        except:
            # Caso não haja mais botão "Mais", encerrar o loop
            break

    return lista_ofertas_google

def buscar_precos_buscape(navegador, nome, termos_banidos, preco_minimo, preco_maximo):
    #Abrir o site do Buscape:
    navegador.get("https://www.buscape.com.br/")
    #Buscar o produto:
    navegador.find_element(By.XPATH, '//*[@id="new-header"]/div[1]/div/div/div[3]/div/div/div[2]/div/div[1]/input').send_keys(nome)
    navegador.find_element(By.CLASS_NAME, 'AutoCompleteStyle_SearchIconWrapper__Knh_x').click()
    #Definir preços:
    preco_minimo = float(preco_minimo)
    preco_maximo = float(preco_maximo)
    #Crias lista de filtros da busca:
    lista_termos_produto = nome.lower().split(" ")
    lista_termos_banidos = termos_banidos.lower().split(" ")

    #Lista que será retornada pela função:
    lista_ofertas_buscape = []

    while True:
        #Esperar até que os resultados da pesquisa estejam disponíveis na página:
        elemento = WebDriverWait(navegador, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'SearchCard_ProductCard_Inner__7JhKb')))
        time.sleep(1)

        #Pegar todos os produtos da página:
        lista_produtos = navegador.find_elements(By.CLASS_NAME, 'SearchCard_ProductCard_Inner__7JhKb')

        for produto in lista_produtos:
            try:
                #Obter os dados dos produtos:
                nome_produto = produto.find_element(By.TAG_NAME, 'h2').text.lower().replace(',','')
                preco_produto = produto.find_element(By.TAG_NAME, 'p').text.replace(' ', '').replace('R$', '').replace('.', '').replace(',', '.')

                #Filtrar pelos dados da busca:
                termo_prod_correto = all(termo in nome_produto for termo in lista_termos_produto)
                termo_banido_encontrado = any(termo in nome_produto for termo in lista_termos_banidos)

                if termo_prod_correto and not termo_banido_encontrado:
                    preco_produto = float(preco_produto)
                    if preco_minimo <= preco_produto <= preco_maximo:
                        preco_produto = formatar_moeda(preco_produto)
                        link_produto = produto.get_attribute('href')
                        lista_ofertas_buscape.append((nome_produto, preco_produto, link_produto))
            except:
                pass
        
        try:
            #Localizar e tentar clicar no botão próxima página:
            lista_paginas = navegador.find_element(By.CLASS_NAME, 'Paginator_paginator__j178K')
            paginas = lista_paginas.find_elements(By.TAG_NAME, 'li')
            proxima_pag = paginas[-1]
            proxima_pag.click()
            
        except:
            #Caso não haja mais o botão, encerrar o loop:
            break
        
    return lista_ofertas_buscape

def concatenar_tabelas(tabela, lista):
    if lista:
        tabela_lista = pd.DataFrame(lista, columns=['Produto', 'Preço', 'Link'])
        tabela = pd.concat([tabela, tabela_lista], ignore_index=True)
        return tabela
    else:
        tabela_lista = None
        return None

In [8]:
#Criar a tabela com os produtos, preços e links para compras encontrados:
tabela_ofertas = pd.DataFrame()

for linha in produtos_df.index:
    nome = produtos_df.loc[linha, 'Nome']
    termos_banidos = produtos_df.loc[linha, 'Termos banidos']
    preco_minimo = produtos_df.loc[linha, 'Preço mínimo']
    preco_maximo = produtos_df.loc[linha, 'Preço máximo']

    lista_ofertas_google_shopping = buscar_precos_google_shopping(navegador, nome, termos_banidos, preco_minimo, preco_maximo)
    
    tabela_ofertas = concatenar_tabelas(tabela_ofertas, lista_ofertas_google_shopping)

    lista_ofertas_buscape = buscar_precos_buscape(navegador, nome, termos_banidos, preco_minimo, preco_maximo)

    tabela_ofertas = concatenar_tabelas(tabela_ofertas, lista_ofertas_buscape)

#Exportar tabela pro excel:
tabela_ofertas.to_excel(r'Listas de Compras\TabelaOfertas.xlsx', index=False)

In [10]:
#Enviar por e-mail o resultado da tabela:

#Verificando se existe alguma oferta dentro da tabela de ofertas
if len(tabela_ofertas.index) > 0:
    #Criar e enviar o e-mail:
    outlook = win32.Dispatch('outlook.application')
    mail = outlook.CreateItem(0)
    mail.To = 'diegomext+compras@gmail.com'
    mail.Subject = 'Produto(s) Encontrado(s) na faixa de preço desejada'
    mail.HTMLBody = f"""
    <p>Prezado,</p>
    <p>Encontramos alguns produtos em oferta dentro da faixa de preço desejada. Segue tabela com detalhes:</p>
    {tabela_ofertas.to_html(index=False)}
    <p>Qualquer dúvida estou à disposição</p>
    <p>Att.,</p>
    <p>Diego Monutti</p>
    """
    mail.Send()

#Fechar o navegador
navegador.quit()